# ARCHON-RH: Complete Colab Setup for Riemann Hypothesis Research
This notebook sets up the complete reasoning lab for formal mathematics and theorem proving.

**Hardware Required:** GPU Runtime (T4/V100/A100)

**Runtime Setup:**
- Runtime → Change runtime type → Hardware accelerator: **GPU**
- Runtime type: **High-RAM** (if available)


## Step 1: Verify GPU & System


In [ ]:
# Check GPU availability and specs
!nvidia-smi
!python -V

import torch
print(f"\n{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
print(f"{'='*60}")


## Step 2: Get Code (Choose ONE option below)


In [ ]:
# OPTION 1: Clone from GitHub (RECOMMENDED)
# First, push your code to GitHub, then replace YOUR_USERNAME below
!rm -rf /content/archon-rh
!git clone https://github.com/YOUR_USERNAME/archon-rh /content/archon-rh
%cd /content/archon-rh
!pwd
!ls -la


In [ ]:
# OPTION 2: Upload ZIP file (Alternative if no GitHub)
# First, create archon-rh.zip locally, then run this cell
from google.colab import files
import zipfile

print("Upload archon-rh.zip containing the archon-rh folder")
uploaded = files.upload()

zip_name = list(uploaded.keys())[0]
!unzip -q {zip_name} -d /content/
%cd /content/archon-rh
!pwd
!ls -la


In [ ]:
# OPTION 3: Mount Google Drive (if code is on Drive)
# Upload archon-rh folder to Google Drive first
from google.colab import drive
drive.mount('/content/drive')

# Update path to where you uploaded the folder
!cp -r /content/drive/MyDrive/archon-rh /content/archon-rh
%cd /content/archon-rh
!pwd
!ls -la


## Step 3: Verify Code is Loaded


In [ ]:
# Verify directory structure
!pwd
!ls -la
print("\n✓ Checking key directories:")
!ls -d prover/ conjecture/ numerics/ orchestration/ 2>/dev/null || echo "⚠️  Missing directories - check your setup!"


## Step 4: Install Dependencies (4-7 minutes)


In [ ]:
# Upgrade pip and install base dependencies
!python -m pip install -U pip setuptools wheel

# Install the package in editable mode with dev dependencies
!pip install -e .[dev]

# Install GPU acceleration libraries
!pip install accelerate bitsandbytes

# Install additional dependencies for better performance
!pip install pandas pyarrow rich

print("\n✓ Installation complete!")


## Step 4: Build Dataset (Mathlib Tactics)


In [ ]:
# Build dataset shard from Mathlib (or use fallback)
from prover.datasets.build_mathlib_dataset import build

print("Building dataset shard (limit=64 examples)...")
shard = build(limit=64)
print(f"✓ Dataset ready with {len(shard.examples)} examples")
print(f"✓ Output: artifacts/data/mini_mathlib.jsonl")

# Verify the dataset
!ls -lh artifacts/data/
!head -n 3 artifacts/data/mini_mathlib.jsonl


## Step 5: Train SFT Model (GPU-Accelerated)
Supervised fine-tuning on tactic prediction.


In [ ]:
# Train model with GPU-optimized config
!python prover/train/sft_hf.py --config orchestration/configs/colab_sft_gpu.yaml

print("\n✓ SFT Training complete!")
!ls -lh artifacts/checkpoints/sft_colab_gpu/


## Step 6: Run PPO Reinforcement Learning
Self-play loop for policy optimization.


In [ ]:
from prover.rl.ppo_loop import train

print("Starting PPO training...")
train('orchestration/configs/colab_rl_gpu.yaml')
print("✓ PPO training complete!")

!ls -lh artifacts/checkpoints/rl_colab_gpu/


## Step 7: FunSearch Conjecture Discovery
Generate and evolve mathematical conjectures.


In [ ]:
# Run FunSearch loop for discovering new conjectures
!python conjecture/funsearch/funsearch_loop.py --config orchestration/configs/funsearch_loop.yaml

print("\n✓ FunSearch complete!")
!ls -lh artifacts/funsearch/


## Step 8: Download All Artifacts


In [ ]:
# Zip and download all results
!zip -r /content/archon_rh_artifacts.zip artifacts/

from google.colab import files
files.download('/content/archon_rh_artifacts.zip')
print("\n✓ All artifacts downloaded as archon_rh_artifacts.zip")
